In [1]:
import pandas as pd
import yfinance as yf
import pymongo
import time
import datetime

from datetime import datetime, timedelta
from pymongo import MongoClient

In [2]:
tickersdf = yf.download("WMT NKLA AMZN DIS FB SPCE BA AMD MSFT AAPL TSLA SPY", start = '2019-10-01', end = '2020-10-03', progress = False)
client = MongoClient('localhost', 27017)
db = client.db
comments = db.comments

commentdf = pd.DataFrame.from_records(comments.find({'created_utc': {'$exists': True}}))
commentdf.drop(commentdf[commentdf['sentiment'] == 0].index, inplace=True)
commentdf.reset_index(drop = True, inplace = True)

In [3]:
spydf = yf.download('SPY', start = '2019-10-01', end = '2020-10-03', progress = False)
dayBins = list(spydf.reset_index()['Date'])
commentdf['created_utc'] = [pd.Timestamp(datetime.utcfromtimestamp(time)) for time in commentdf['created_utc']]

In [4]:
commentdf['days'] = pd.cut(commentdf['created_utc'], bins = dayBins, labels=spydf.reset_index()['Date'][0:254])

In [5]:
data = pd.pivot_table(commentdf, values = 'id', index = ['days'], columns = ['sentiment'], aggfunc = 'count')
data.columns.name = ''
data.reset_index(inplace= True)

In [6]:
tickersdf = tickersdf.stack().swaplevel(0,1).drop(columns = ['Close', 'High', 'Low', 'Volume','Adj Close'])

In [7]:
data['tot'] = data['bullish'] + data['bearish']
data['avg'] = (data['bullish']/data['tot']).rolling(5).mean()
data['days'] = data['days'].astype('datetime64[ns]')

uploaddf = tickersdf.loc['SPY'].merge(data, left_index = True, right_on = 'days')
uploaddf['days'] = uploaddf['days'].apply(lambda x: x.value)
uploaddf.set_index('days', inplace = True)

In [8]:
lineDict = {}

In [9]:
lineDict['SPY'] = uploaddf.to_dict('index')

In [10]:
len(lineDict['SPY'])

254

In [11]:
stocklist = ['WMT', 'NKLA', 'AMZN', 'DIS', 'FB', 'SPCE', 'BA', 'AMD', 'MSFT', 'AAPL', 'TSLA']

In [12]:
for nextStock in stocklist:
    print(nextStock)
    newdatadf = commentdf.loc[pd.DataFrame(commentdf['stocks'].tolist()).isin([nextStock]).values]

    data = pd.pivot_table(newdatadf, values = 'id', index = ['days'], columns = ['sentiment'], aggfunc = 'count')
    data.columns.name = ''
    data.reset_index(inplace= True)

    data['tot'] = data['bullish'] + data['bearish']
    data['avg'] = (data['bullish']/data['tot']).rolling(3).mean()
    data['days'] = data['days'].astype('datetime64[ns]')

    data.fillna(0, inplace=True)

    data['tot'] = data['bullish'] + data['bearish']
    data['avg'] = (data['bullish']/data['tot']).rolling(3).mean()
    data['days'] = data['days'].astype('datetime64[ns]')

    uploaddf = tickersdf.loc[nextStock].merge(data, left_index = True, right_on = 'days', how='left')
    uploaddf['days'] = uploaddf['days'].apply(lambda x: x.value)
    uploaddf.set_index('days', inplace = True)

    lineDict[nextStock] = uploaddf.to_dict('index')

WMT
NKLA
AMZN
DIS
FB
SPCE
BA
AMD
MSFT
AAPL
TSLA


In [13]:
lineDict.keys()

dict_keys(['SPY', 'WMT', 'NKLA', 'AMZN', 'DIS', 'FB', 'SPCE', 'BA', 'AMD', 'MSFT', 'AAPL', 'TSLA'])

In [14]:
"WMT NKLA AMZN DIS FB SPCE BA AMD MSFT AAPL TSLA SPY"

'WMT NKLA AMZN DIS FB SPCE BA AMD MSFT AAPL TSLA SPY'

In [15]:
import json
with open('result.json', 'w') as fp:
    json.dump(lineDict, fp)

# Function needed to convert this output to usable output in lineBar.js
/*var stockdata = [];
    for (var i = 0; i < tickers.length; i ++){
        var ticker = tickers[i]
        indTicker = data[ticker]
        indTicker = d3.map(indTicker).entries()
        indTicker.forEach(function(d) {
            d.key = parse(d.key * 10**-9)
        })
        stockdata.push({
            name: ticker,
            values: indTicker.map(function(entry) {
                return{time: entry.key, Open: +entry.value.Open, 
                       bullish: +entry.value.bullish, bearish: +entry.value.bearish, 
                       neutral: +entry.value.neutral, tot: +entry.value.tot, 
                       avg: +entry.value.avg}
            })
        })
    }*/

In [24]:
import re
import nltk
from nltk.corpus import stopwords
from operator import itemgetter
nltk.download('stopwords')
wordstop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def cleanStrings(string):
    return re.sub("[^a-zA-Z0-9./$:,'&]+", ' ',string) #only include normal string characters
def cleanText(text):
    return re.sub("http[s]?://\S+", ' ', text) #Remove links

In [5]:
def count(elements):
    if elements[-1] == '.':
        elements = elements[0:len(elements) - 1]

    if elements in corpusdict:
        corpusdict[elements] += 1
    
    else:
        corpusdict.update({elements: 1})

In [6]:
commentdf['body'] = commentdf['body'].apply(cleanStrings)
commentdf['body'] = commentdf['body'].apply(cleanText)

In [25]:
tags = ['bullish', 'bearish', 'neutral']
dataDict = {}
resultsdict = {}

In [66]:
for ticker in ["WMT", "NKLA", "AMZN", "DIS", "FB", "SPCE", "BA", "AMD", "MSFT", "AAPL", "TSLA", "SPY"]:
    mask = commentdf.stocks.apply(lambda x: ticker in x)
    indtickdf = commentdf[mask]
    dataDict = {}
    for tag in ['bullish', 'bearish', 'neutral']:
        print(tag)
        corpusdict = {}
        for body in indtickdf.loc[commentdf['sentiment'] == tag, 'body']:
            lst = body.lower().split()
            for elements in lst:
                count(elements)
        keepkeys = set(corpusdict.keys()).difference(wordstop)
        corpusdict = {keepkey: corpusdict[keepkey] for keepkey in keepkeys}
        N = 75
        topN = dict(sorted(corpusdict.items(), key = itemgetter(1), reverse = True)[:N])
        resultlist = []
        for key, value in topN.items():
            resultlist.append({'text': key, 'size': value})
        dataDict[tag] = resultlist
    resultsdict[ticker] = dataDict

bullish
bearish
neutral
bullish
bearish
neutral
bullish
bearish
neutral
bullish
bearish
neutral
bullish
bearish
neutral
bullish
bearish
neutral
bullish
bearish
neutral
bullish
bearish
neutral
bullish
bearish
neutral
bullish
bearish
neutral
bullish
bearish
neutral
bullish
bearish
neutral


In [75]:
dataDict = {}
for tag in ['bullish', 'bearish', 'neutral']:
    print(tag)
    corpusdict = {}
    for body in commentdf.loc[commentdf['sentiment'] == tag, 'body']:
        lst = body.lower().split()
        for elements in lst:
            count(elements)
    keepkeys = set(corpusdict.keys()).difference(wordstop)
    corpusdict = {keepkey: corpusdict[keepkey] for keepkey in keepkeys}
    N = 250
    topN = dict(sorted(corpusdict.items(), key = itemgetter(1), reverse = True)[:N])
    resultlist = []
    for key, value in topN.items():
        resultlist.append({'text': key, 'size': value})
    dataDict[tag] = resultlist
resultsdict['ALL'] = dataDict

bullish
bearish
neutral


In [76]:
with open('results.json', 'w') as fp:
    json.dump(resultsdict, fp)